In [ ]:
import pandas as pd
import warnings
import pickle
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier

from model import SmokerPrediction

df = pd.read_csv(r"smoking.csv")

In [ ]:
#RandomForestClassifier
sp1 = SmokerPrediction(df, False)

df5 = sp1.create_features()

X_train, X_val, y_train, y_val = sp1.scaling(df5, False, False, False, 0, RandomForestClassifier)

model, y_pred_rf, threshold = sp1.train_xgboost_mcc_optimized(X_train, y_train, X_val, y_val, RandomForestClassifier)

importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

y_pred_rf = (model.predict_proba(X_val) >= threshold)[:,1].astype(int)


RandomForestClassifier: {'n_estimators': 275, 'max_depth': 23,'min_samples_split':30,'min_samples_leaf':10,'max_features':'sqrt','random_state':42,'n_jobs':-1,'max_samples':0.75, 'ccp_alpha':0.000075}

In [ ]:
#RandomForest
df_feat_rf = SmokerPrediction(df, drop=False).create_features()

X = df_feat_rf.drop("smoking", axis=1)
y = df_feat_rf["smoking"]

# 3. Train ONE final model (best params you found)
final_model = RandomForestClassifier(n_estimators = 275, max_depth = 23, min_samples_split = 30, min_samples_leaf = 10, max_features = 'sqrt', random_state = 42, n_jobs = -1, max_samples = 0.75, ccp_alpha = 0.000075)

final_model.fit(X, y)

# 4. Save EVERYTHING needed for inference
artifact = {
    "model": final_model,
    "features": X.columns.tolist(),
    "threshold": 0.5
}

with open("smoker_model_rf.pkl", "wb") as f:
    pickle.dump(artifact, f)